In [ ]:
import numpy as np
import trident
import yt
import os
import sys
from multiprocessing import Pool,current_process,cpu_count
import itertools
import logging
import matplotlib.pyplot as plt
from quasar_scan import *
from parse_vela_metadata import Rdict, Ldict

In [4]:
class Multi_Quasar_Sphere_Analyzer(QuasarSphere):
    #either load all quasar spheres, load by VELA_SIM, or load by redshift
    #def _init_(self, loadAll = False, VELA_SIM = "", redshift = -1):
    
    def _init_(self):
        pass
        
    def graph_by_rVir(q1):
        x = np.linspace(0,scanparams[4],scanparams[3])
        print(x)
        y = []
    
        
    #def graph_by_VELA_galaxy():
        
    #def graph_by_redshift():
        
        




qMany = Multi_Quasar_Sphere_Analyzer()

continuedfilename = '/global/homes/b/bryanw/quasarscan/output/VELA_v2_01coldensinfo/406_of_400-_OVI_NeVIII_HI_CIII_OIV_NIII_MgII_OV_OIII_NIV_MgX_NV_SIV_OII_SIII_SII_SV_SVI_NII_z0.99.txt'
simparams,scanparams,ions,data = read_values(continuedfilename)

q = QuasarSphere(simparams = simparams,scanparams = scanparams,ions = ions ,data = data, readOnly = True)

qMany.graph_by_rVir(q)


In [ ]:
from quasar_scan import *
from parse_vela_metadata import Rdict, Ldict

dataset = '/global/homes/b/bryanw/VELA05v2/10MpcBox_csf512_a0.500.d'

sim_name = "VELA_v2_05"
aExpansion = '0.500'

#use the function to get L and Rvir

#Rvir = float(Rdict["VELA27"]["0.200"])
#Lstr = Ldict["VELA27"]["0.200"]
#L = np.zeros(3)
#for i in 0, 1, 2:
#    L[i] = float(Lstr[i])
    
Rvir, L = get_vela_metadata(sim_name, aExpansion)

dictionary = parse_vela_metadata.dict_of_vela_info("Rvir",iarr = np.arange(1,35))

### THIS IS THE VERSION WITHOUT L
#rvir = float(dictionary[sim_name][aExpansion])
#q = QuasarSphere(ions = '[O VI, Ne VIII, H I, C III, O IV, N III, Mg II, O V, O III, N IV, Mg X, N V, S IV, O II, S III, S II, S V, S VI, N II]', simname = sim_name, dspath = ds, Rvir = rvir)
### THIS IS THE VERSION WITH L 
q = Multi_Quasar_Sphere_Analyzer(ions = '[O VI, Ne VIII, H I, C III, O IV, N III, Mg II, O V, O III, N IV, Mg X, N V, S IV, O II, S III, S II, S V, S VI, N II]', simname = sim_name, dspath = dataset, Rvir = Rvir, L = L)



In [8]:
qmany = Multi_Quasar_Sphere_Analyzer()

UnboundLocalError: local variable 'convert' referenced before assignment